# **추출 체인(Extraction Chain) 구축하기**

이 노트북에서는 **채팅 모델(Chat Models)** 의 **도구 호출(Tool Calling)** 기능을 사용하여 **비정형 텍스트에서 구조화된 정보를 추출**하는 방법을 다룹니다. 또한 이 맥락에서 **Few-Shot 프롬프팅(Few-Shot Prompting)** 을 사용하여 성능을 개선하는 방법을 시연할 것입니다.

### 주요 내용
- Pydantic을 활용해 데이터 추출을 위한 스키마(Schema) 정의  
- LangChain의 Tool Calling 기능을 사용해 LLM이 구조화된 데이터를 반환하도록 설정  
- Few-Shot 프롬프팅(Few-Shot Prompting) 기법을 사용해 성능 향상  
- 다중 엔터티(Multiple Entity) 추출 지원 (여러 개의 인물 정보 추출 가능)

## **스키마 (The Schema)**  

먼저, 텍스트에서 어떤 정보를 추출할 것인지 **정의**해야 합니다.  

이를 위해 **Pydantic**을 사용하여 **개인 정보(personal information)** 를 추출하기 위한 예제 **스키마(schema)** 를 정의할 것입니다.

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from typing import Optional  
from pydantic import BaseModel, Field 

class Person(BaseModel):
    """
    사람에 대한 정보.
    - 모든 필드는 선택적(optional)입니다.  
    - 각 필드는 `description`을 포함하며, LLM이 이를 활용하여 정확한 추출 결과를 생성합니다.
    """

    name: Optional[str] = Field(default=None, description="사람의 이름")
    hair_color: Optional[str] = Field(default=None, description="사람의 머리 색상")
    height_in_meters: Optional[str] = Field(
        default=None, description="미터 단위로 측정된 키"
    )

## **스키마 정의의 두 가지 사례**

1. **속성(attributes)** 과 **스키마(schema)** 를 Pydantic으로 문서화  
   - 이 정보는 LLM에 전달되며, Pydantic 스키마를 통해 명확하게 정의되어 정보 추출의 품질을 개선하는 데 사용됩니다.
     
<pr></pr>

2. **LLM이 정보를 지어내지 않도록 합니다.**  
   - 각 속성에 `Optional`을 사용하여 LLM이 답을 모를 경우 `None`을 반환할 수 있도록 했습니다.
  
최상의 성능을 얻으려면 **스키마를 잘 문서화**하고, 텍스트에 추출할 정보가 없을 경우 모델이 결과를 **강제로 반환하지 않도록** 설정합니다.  


## **추출기 (The Extractor)**

이제 위에서 정의한 **스키마(schema)** 를 사용하여 **정보 추출기(Information Extractor)** 를 만들어 봅니다.

In [3]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# 사용자 정의 프롬프트 템플릿 정의
# 텍스트에서 정보를 추출하기 위한 명확한 지침과 추가 컨텍스트를 제공합니다.
prompt_template = ChatPromptTemplate(
    [
        ("system",
            "당신은 전문 정보 추출 알고리즘입니다. "
            "텍스트에서 관련 정보만 추출하세요. "
            "추출해야 할 속성의 값을 알지 못할 경우, "
            "해당 속성의 값으로 null을 반환하세요."
            "한국어로 반환하세요."
        ),
        ("user", "{text}"),   # 사용자 입력 텍스트를 프롬프트에 전달
    ]
)

**기능/도구 호출(Function/Tool Calling)** 을 지원하는 모델을 사용해야 합니다.

In [4]:
from langchain.chat_models import init_chat_model

model = init_chat_model("gpt-5-nano", model_provider="openai")
# model = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

In [5]:
# LLM에서 구조화된 출력을 생성하도록 스키마 바인딩
structured_llm = model.with_structured_output(schema=Person)
structured_llm

RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x10f7e84d0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x10f8f2050>, root_client=<openai.OpenAI object at 0x10f052450>, root_async_client=<openai.AsyncOpenAI object at 0x10f8f1d50>, model_name='gpt-5-nano', model_kwargs={}, openai_api_key=SecretStr('**********')), kwargs={'response_format': <class '__main__.Person'>, 'ls_structured_output_format': {'kwargs': {'method': 'json_schema', 'strict': None}, 'schema': {'type': 'function', 'function': {'name': 'Person', 'description': '사람에 대한 정보.\n- 모든 필드는 선택적(optional)입니다.  \n- 각 필드는 `description`을 포함하며, LLM이 이를 활용하여 정확한 추출 결과를 생성합니다.', 'parameters': {'properties': {'name': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': None, 'description': '사람의 이름'}, 'hair_color': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': None, 'description': '사람의 머리 색상'}, 'height_in_meters'

In [6]:
text = "Smith의 키는 186센티이고 금발입니다."

prompt = prompt_template.invoke({"text": text})
response = structured_llm.invoke(prompt)
response.model_dump()

{'name': 'Smith', 'hair_color': '금발', 'height_in_meters': '1.86'}

LLM은 생성 모델이므로, 센티미터로 제공된 신장의 정보를 미터로 정확하게 추출하는 등의 놀라운 작업을 수행할 수 있습니다! 또한 스키마에 정의된 이름, 머리색, 키 외의 다른 내용은 무시하고 답변을 생성합니다.


## 다중 엔터티

많은 경우, 단일 엔티티가 아닌 여러 엔티티를 추출해야 합니다. 이는 Pydantic에서 모델을 서로 중첩하여 쉽게 구현할 수 있습니다.

In [7]:
from typing import List

class Data(BaseModel):
    """
    여러 사람들에 대한 추출된 데이터.
    """
    peoples: List[Person]   # 여러 사람의 정보를 추출하기 위해 'Person' 모델의 리스트를 정의

In [8]:
structured_llm = model.with_structured_output(schema=Data)

text = "제 이름은 제프이고, 제 머리는 검은색이고 키는 6피트입니다. 안나는 저와 같은 색의 머리를 가지고 있습니다."

prompt = prompt_template.invoke({"text": text})
response = structured_llm.invoke(prompt)
response.model_dump()

{'peoples': [{'name': '제프', 'hair_color': '검은색', 'height_in_meters': '1.83'},
  {'name': '안나', 'hair_color': '검은색', 'height_in_meters': None}]}

**여러 엔티티**를 추출할 수 있도록 스키마가 설계되면, 텍스트에 관련 정보가 없을 경우 **빈 리스트(empty list)** 를 반환하여 **아무런 엔티티도 추출하지 않을 수 있습니다.**  

이는 일반적으로 **좋은 설계**입니다! 이를 통해 모델이 해당 엔티티를 반드시 감지하도록 강제하지 않을 수 있습니다.  

In [9]:
text = "하늘은 파랗고 나무는 푸르다."

prompt = prompt_template.invoke({"text": text})
dict(structured_llm.invoke(prompt))

{'peoples': []}

## 구조화된 출력 (Structured Outputs)과 함수 호출 (Pydantic 기반)

일반적인 챗봇은 모델이 **자연어로 직접 응답**하지만,  
데이터 분석·API 호출·데이터베이스 저장 같은 상황에서는  
모델이 **정해진 구조(스키마)** 에 맞게 출력을 내보내야 합니다.  
이를 **구조화된 출력(Structured Output)** 이라고 합니다.

---

### 핵심 포인트

- **자연어 응답이 아닌 구조화된 데이터(JSON 형태)** 로 응답.  
- **데이터베이스, API, 파이프라인** 등과의 호환성 보장.  
- **Pydantic 스키마**로 검증된 일관된 데이터 형식 확보.

---

### 구조화된 출력 + 함수 호출 예제 (Pydantic 기반)

1. **Pydantic 스키마 정의**  
   모델이 따라야 할 출력 구조를 정의합니다.
   ```python
   class GeoQuery(BaseModel):
       """위도와 경도 정보를 포함한 위치 데이터"""
       latitude: float = Field(..., description="위도 (−90 ~ 90)")
       longitude: float = Field(..., description="경도 (−180 ~ 180)")


In [10]:
from typing import Optional
import requests
from pydantic import BaseModel, Field, field_validator

# 외부 API 호출 함수
def get_temperature(latitude: float, longitude: float) -> float:
    """Open-Meteo에서 현재 기온(°C)을 조회"""
    url = (
        f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m"
    )
    resp = requests.get(url, timeout=10)
    resp.raise_for_status()
    data = resp.json()
    return float(data["current"]["temperature_2m"])

In [11]:
# Pydantic 스키마 정의 (LLM이 이 스키마를 참고해 JSON 생성 -> 파이썬에서 검증/파싱)
class GeoQuery(BaseModel):
    """
    사용자가 지정한 위치의 좌표.
    - 모든 값은 십진수 좌표이며, 범위 검증을 수행합니다.
    - 예) 서울: (37.5665, 126.9780)
    """
    latitude: float = Field(..., description="위도 (−90 ~ 90)")
    longitude: float = Field(..., description="경도 (−180 ~ 180)")

llm_with_structure = model.with_structured_output(schema=GeoQuery)

user_input = "서울의 현재 날씨를 알려줘."

# 좌표 추출 (LLM이 JSON 생성 -> Pydantic이 객체화)
# coords는 GeoQuery 인스턴스 (예: GeoQuery(latitude=37.5665, longitude=126.9780))
coords: GeoQuery = llm_with_structure.invoke(user_input)

# 도구(외부 API) 호출
temperature = get_temperature(coords.latitude, coords.longitude)

# 최종 응답 생성
final_prompt = (
    f"현재 서울(위도 {coords.latitude}, 경도 {coords.longitude})의 기온은 "
    f"{temperature}°C입니다. 사용자가 이해하기 쉽게 자연스러운 한국어로 한 문단으로 답변하세요."
)
final_response = model.invoke(final_prompt)

# 결과 출력
print("사용자 질문:", user_input)
print("추출 좌표:", coords.model_dump())
print("현재 기온(°C):", temperature)
print("LLM이 생성한 최종 응답:", final_response.content)

사용자 질문: 서울의 현재 날씨를 알려줘.
추출 좌표: {'latitude': 37.5665, 'longitude': 126.978}
현재 기온(°C): 24.5
LLM이 생성한 최종 응답: 현재 서울의 기온은 24.5°C로 밖에 나가기에 꽤 쾌적한 날씨예요. 가볍게 반팔 차림으로도 충분하지만 햇빛이 강할 수 있으니 야외 활동 시 자외선 차단제와 모자, 얕은 겉옷 정도를 챙기는 게 좋습니다. 한낮에는 조금 더울 수 있으니 물을 자주 마시고, 땀을 많이 흘리면 실내로 들어가 휴식을 취하는 것도 좋습니다.


--------

### 실습 문제: Open-Meteo API를 이용한 실제 날씨 정보 조회 

**목표:**
사용자가 "xx 도시의 날씨를 알려줘"라고 입력하면 모델이 위도/경도를 추론하고, `get_weather(latitude, longitude)` 함수를 호출한 뒤, 결과를 출력하도록 하세요.